# This part of the lab was submitted in MySQL #


USE sakila;


CREATE TABLE rentals_may AS
SELECT *
FROM rental
WHERE MONTH(rental_date) = 5 AND YEAR(rental_date) = 2005;  -- Assuming we're interested in May 2005



select * from sakila.rentals_june;


INSERT INTO rentals_may
SELECT *
FROM rental
WHERE MONTH(rental_date) = 5 AND YEAR(rental_date) = 2005; 

CREATE TABLE rentals_june AS
SELECT *
FROM rental
WHERE MONTH(rental_date) = 6 AND YEAR(rental_date) = 2005; 


INSERT INTO rentals_june
SELECT *
FROM rental
WHERE MONTH(rental_date) = 6 AND YEAR(rental_date) = 2005; 


SELECT customer_id, COUNT(*) as rental_count
FROM rentals_may
GROUP BY customer_id
ORDER BY rental_count DESC;

# Creating a Python connection with SQL database and retrieving the results of the last two queries as dataframes #

In [1]:
pip install pymysql pandas

  Obtaining dependency information for pymysql from https://files.pythonhosted.org/packages/e5/30/20467e39523d0cfc2b6227902d3687a16364307260c75e6a1cb4422b0c62/PyMySQL-1.1.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/44.8 kB ? eta -:--:--
   ------------------ --------------------- 20.5/44.8 kB 330.3 kB/s eta 0:00:01
   --------------------------- ------------ 30.7/44.8 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 44.8/44.8 kB 244.9 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine

user = 'root'
password = 'wailbak123xxx'
host = 'localhost'
database = 'sakila'

connection_url = f'mysql+pymysql://{user}:{password}@{host}/{database}'

engine = create_engine(connection_url)

query_may = """
SELECT customer_id, COUNT(*) AS rental_count_may
FROM rentals_may
GROUP BY customer_id
ORDER BY rental_count_may DESC;
"""

query_june = """
SELECT customer_id, COUNT(*) AS rental_count_june
FROM rentals_june
GROUP BY customer_id
ORDER BY rental_count_june DESC;
"""

df_rentals_may = pd.read_sql_query(query_may, engine)
df_rentals_june = pd.read_sql_query(query_june, engine)

In [11]:
df_rentals_may.head()

,customer_id,rental_count_may
0,197,16
1,506,14
2,109,14
3,269,12
4,239,12


In [13]:
df_rentals_june.head()

,customer_id,rental_count_june
0,31,22
1,454,20
2,329,18
3,295,18
4,561,18


In [16]:
def compare_rentals(df_may, df_june):
    # Merging the two dataframes on 'customer_id'
    df_merged = pd.merge(df_may, df_june, on='customer_id', how='outer').fillna(0)
    
    # Adding a new column to determine if more or less films were borrowed in June compared to May
    df_merged['comparison'] = df_merged.apply(lambda row: 'More in June' if row['rental_count_june'] > row['rental_count_may'] else ('Less in June' if row['rental_count_june'] < row['rental_count_may'] else 'Equal'), axis=1)
    
    return df_merged

df_comparison = compare_rentals(df_rentals_may, df_rentals_june)

df_comparison


,customer_id,rental_count_may,rental_count_june,comparison
0,197,16.0,16.0,Equal
1,506,14.0,10.0,Less in June
2,109,14.0,10.0,Less in June
3,269,12.0,6.0,Less in June
4,239,12.0,10.0,Less in June
...,...,...,...,...
593,63,0.0,2.0,More in June
594,598,0.0,2.0,More in June
595,487,0.0,2.0,More in June
596,555,0.0,2.0,More in June
